In [1]:
# Author: Zhengxiang (Jack) Wang 
# Date: 2021-08-02
# GitHub: https://github.com/jaaack-wang 
# About: Word Vectors and Word Senses for Stanford CS224N- NLP with Deep Learning | Winter 2019

# Table of Contents
- [1. Casual takeaways](#1)
- [2. Word2vec](#2)
    - [2.1 Recap](#2-1)
    - [2.2 Two model variants](#2-2)
    - [2.3 The skip-gram model with negative sampling (HW2)](#2-3)
        - [2.3.1 Negative sampling](#2-3-1)
        - [2.3.2 Objective/loss functions](#2-3-2)
        - [2.3.3 Subsampling](#2-3-3)
- [3. Co-occurrence counts based method](#3)
    - [3.1 Co-occurrence matrix](#3-1)
    - [3.2 Dimensionality reduction](#3-2)
        - [3.2.1 Classic Method: Singular Value Decomposition](#3-2-1)
        - [3.2.2 Other tricks: scaling, pearson correlations etc.](#3-2-2)
    - [3.3 Count based method vs. direct prediction (neural network) method](#3-3)
- [4. GloVe: Global Vectors for Word Representation](#4)
    - [4.1 Basic idea](#4-1)
    - [4.2 Mathematical realization ](#4-2)
- [5. Evaluation of word vectors](#5)
    - [5.1 Overview](#5-1)
    - [5.2 Intrinsic word vector evaluation](#5-2)
        - [5.2.1 Models](#5.2.1)
        - [5.2.2 Performances of different models](#5-2-2)
    - [5.3 Extrinsic word vector evaluation](#5-3)
    - [5.4 Factors that impact the performances](#5-4)
- [6. Challenges: word sense ambiguity](#6)
    - [6.1 Challenges](#6-1)
    - [6.2 Tentative solutions](#6-2)
- [7. References](#7)

<a name='1'></a>
# 1. Casual takeaways

- For Word2vec paper, the paper looks farily clean, but in the source code is filled with plenty tricks that help improve the performance but are not particularly theoretical. -- Very typical of deep learning actually! Experiment with hyperparameters and choose one set that seems to work best. 
- Co-occurrence counts based method is a traditional way to represent the distributed words before 2013 when neural network based method took off.
- Counts-based method has been explored more in COG psych than anywhere else. 
- GloVe performances well on several word vectors benchmarks because "that is our paper". LOL. 

<a name='2'></a>
# 2. Word2vec


<a name='2-1'></a>
## 2.1 Recap

<font color='blue'>**Previous notes (e.g., the prediction function, the loss functon, how to derive gradients) on Word2vec can be seen in the third session of the lecture 1 notes.**</font>


<img src='../images/2-word2vec-recap.png' width='600' height='300'>

<br>

<img src='../images/2-word2vec-recap2.png' width='600' height='300'>

<br>

**Objective/loss/cost function**

For each position $t = 1, ... , 𝑇$, predict context words within a window of fixed size $m$, given center word $w_j$. Data likelihood:

$$Likelihood = L(\theta) = \prod_{t}^{T} \prod_{\substack{-m \leq j \leq m \\ j \neq m}} P(w_{t+j}|{w_{t}; \theta}) \tag{1}$$

The objective/loss/cost function for $(1)$ is the average negative log likelihood: 

$$J(\theta) = - \frac{1}{T} logL(\theta) = - \frac{1}{T} \sum_{t=1}^{T} \sum_{\substack{-m \leq j \leq m \\ j \neq m}} log P(w_{t+j}|{w_{t}; \theta}) \tag{2}$$


**Prediction function**

Denote by $v_{c}$ and $v_{o}$ respectively the center word and the context word, using **softmax**, we get the following prediction function of predicting $v_{c}$ given $v_{o}$ and the vocubulary $V$:

$$P(o|c) = \frac{exp(\mathbf{u_{o}^{T} v_{c}})}{\sum_{w \in V} exp(\mathbf{u_{w}^{T} v_{c}})} \tag{3}$$


**Gradient descent**

See profs on the notes for the lecture 1. 

With regard to $v_{c}$:

$$\frac{\partial}{\partial \mathbf{v_c}} (- \frac{1}{T}(\mathbf{u_{o}^{T} v_{c}}- \log \sum_{w \in V} exp(\mathbf{u_{w}^{T} v_{c}}))) = - \frac{1}{T}(\mathbf{u_o} - \sum_{x=1}^{V} P(x|c) \cdot  \mathbf{u_{x}}) \tag{4}$$

With regard to $u_{o}$:

$$\frac{\partial}{\partial \mathbf{u_o}} (- \frac{1}{T}(\mathbf{u_{o}^{T} v_{c}}- \log \sum_{w \in V} exp(\mathbf{u_{w}^{T} v_{c}}))) = - \frac{1}{T}(\mathbf{u_o} - \sum_{x=1}^{V} P(x|c) \cdot  \mathbf{v_{c}}) \tag{5}$$

<br>

Due to the large size of the word embedding matrix, we usually use Mini-batch Gradient Descent in which we update the parameters (i.e., word vectors) every $k$ examples ($k$ usually equals 16, 32, 64, $2^n$). In the lecture2, Manning specifically emphasized Stochastic Gradient Descent, which is a special case of Mini-batch Gradient Descent when $k = 1$. Batch Gradient Descent can be seen as Mini-batch Gradient Descent when $k=V$ (vocabulary size).


<img src='../images/2-word2vec-recap3.png' width='600' height='300'>

<br>

<img src='../images/2-word2vec-recap4.png' width='600' height='300'>


<a name='2-2'></a>
## 2.2 Two model variants

- 1. Skip-grams (SG) <br>
Predict context (“outside”) words (position independent) given center word
- 2. Continuous Bag of Words (CBOW) <br>
Predict center word from (bag of) context words

**Two related papers**
- [Mikolov et. al. (2013a). Efficient estimation of word representations in vector space](https://arxiv.org/pdf/1301.3781.pdf)  
- [Mikolov et. al. (2013b). Distributed representation of words and phrases and their compositionality](https://papers.nips.cc/paper/2013/file/9aa42b31882ec039965f3c4923ce901b-Paper.pdf)

<a name='2-3'></a>
## 2.3 The skip-gram model with negative sampling (HW2)

<font color='blue'>A step-by-step implementation of the skip-gram model with negative sampling can be seen [here](https://github.com/jaaack-wang/dl-nlp-using-paddlenlp/blob/main/paddlenlp_updated_notes_English/WordEmbedding/4-training%20word%20embeddings%20using%20skip-gram%20with%20negative%20sampling%20in%20paddle.ipynb).</font>

<a name='2-3-1'></a>
### 2.3.1 Negative sampling
- see [Mikolov et. al. (2013b)](https://papers.nips.cc/paper/2013/file/9aa42b31882ec039965f3c4923ce901b-Paper.pdf)
- Manning: "Focus on **naïve softmax** (simpler, but expensive, training method)"
- Main idea: train binary logistic regressions for a true pair (center word and a word in its context window) versus several noise pairs (the center word paired with a random word)
- More details: Sample certain number of non-context words from the vocabulary and transform activition function for the output layer into sigmoid function: the prediction now becomes predicting whether a given word is a context word or just a negative example (versus calculating the probabilities of all words as the context words)


<a name='2-3-2'></a>
### 2.3.2 Objective/loss functions

Notation used by [Mikolov et. al. (2013b)](https://papers.nips.cc/paper/2013/file/9aa42b31882ec039965f3c4923ce901b-Paper.pdf):

$$J(\theta) = \frac{1}{T} \sum_{t=1}^{T} J_{t}{(\theta)}$$

where

$$J_{t}{(\theta)} = \log \sigma(\mathbf{u_{o}^{T} v_{c}}) + \sum_{i=1}^{k}\mathbb{E}_{j} \sim P(w)[\log \sigma (-\mathbf{u_{j}^{T} v_{c}})]$$

Notation used by Manning:

$$J_{neg-sample}(\mathbf{o, v_{c}, U}) = - \log \sigma(\mathbf{u_{o}^{T} v_{c}}) - \sum_{i=1}^{k} \log \sigma(\mathbf{u_{k}^{T} v_{c}}) \tag{6}$$


Notes: 
- $k$: number of negative samples


- $\sigma$ simgoid function: $\sigma(x) = \frac{1}{1 + e^{-x}}$, which ranges from (0, 1) exclusive with midpoint = $\frac{1}{2}$ when $x=0$, commonly used for binary classification.


- $\mathbf{U}$ the negative words' embedding matrix = $\sum_{i=1}^{k} \mathbf{u_{k}}$


- Objectives: Maximize probability that real outside word appears; minimize probability that random words appear around center word

<a name='2-3-3'></a>
### 2.3.3 Subsampling 

- The main purpose of subsampling is to reduce the frequency of frequent words (e.g., the, of) that does not give too much discrimant semnatic information.

Formula used by Manning:

$$P(w) = \frac {U(w)^{\frac{3}{4}}}{Z} \tag{7}$$

meaning: 
- P(w): the probablity of sampling a word out.


- the unigram distribution U(w) raised to the 3/4 power


- $Z$ normalization term = $\sum_{w \in V} U(w)^{\frac{3}{4}}$


- The equation will sample out frequent words to certain degrees and keep rare words as much as possible. 


<font color='red'>Not so clear how this works. I may misunderstand $Z$. The explanation given by the course note is as follows: is: 0.93/4 = 0.92; Constitution: 0.093/4 = 0.16; bombastic: 0.013/4 = 0.032. Therefore, $U(w)^{\frac{3}{4}}$ has the power to sample more less frequent words.</font>



<br>

Another sumsampling formula proposed by [Mikolov et. al. (2013b)](https://papers.nips.cc/paper/2013/file/9aa42b31882ec039965f3c4923ce901b-Paper.pdf): 

$$P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}}$$

where $f(w_i)$ is the frequency of word $w_i$ (divided by the size of the corpus) and t is a chosen threshold, typically around $10^{-5}$. This formula will aggressively subsample words whose frequency is greater than $t$ and keep those whose frequency is lower than $t$. Example: suppose $f(w_{the}) = 0.05$ and $t = 10^{-5}$, then $P(w_{the}) = 1 - \sqrt{\frac{10^{-5}}{0.05}} \approx 0.986$, so 98.6% of the word "the" will be **sampled out**. Anotehr example: suppose $f(w_{aforementioned}) = 10^{-5}$ and $t = 10^{-5}$, then $P(w_{the}) = 1 - \sqrt{\frac{10^{-5}}{10^{-5}}} = 0.$, so 100% of the word "aforementioned" will be **kept**. 

<br>

**Apparently, Mikolov's subsampling algorithm is much more aggressive.**

<a name='3'></a>
# 3. Co-occurrence counts based method


<a name='3-1'></a>
## 3.1 Co-occurrence matrix

Building a co-occurrence matrix X:

- 2 options: windows vs. full document
- Window: Similar to word2vec, use window around each wordàcaptures some syntactic and semantic information
- Word-document co-occurrence matrix will give general topics (all sports terms will have similar entries) leading to “Latent Semantic Analysis”

Example: 

<img src='../images/2-cooccurrence-matrix-example.png' width='600' height='300'>

<br>

**Problems**:

Simple count co-occurrence vectors
- Vectors increase in size with vocabulary
- Very high dimensional: require a lot of storage (though sparse)
- Subsequent classification models have sparsity issuesàModels are less robust


<a name='3-2'></a>
## 3.2 Dimensionality reduction

**Basic ideas**
- store “most” of the important information in a fixed, small number of dimensions: a dense vector
- Usually 25–1000 dimensions, similar to word2vec


<a name='3-2-1'></a>
### 3.2.1 Classic Method: Singular Value Decomposition

[Singular Value Decomposition](https://en.wikipedia.org/wiki/Singular_value_decomposition)
[A tutorial on Singular Value Decomposition](https://davetang.org/file/Singular_Value_Decomposition_Tutorial.pdf)

- **Running an SVD on raw counts doesn’t work well.**
- SVD based methods do not scale well for big matrices and it is hard to incorporate new words or documents. Computational cost for a m × n matrix is O(mn2)

<br>

<img src='../images/2-svd.png' width='600' height='300'>

<br>

Problems:

- The dimensions of the matrix change very often (new words are added very frequently and corpus changes in size).
- The matrix is extremely sparse since most words do not co-occur.
- The matrix is very high dimensional in general (≈ 106 × 106)
- Quadratic cost to train (i.e. to perform SVD)
- Requires the incorporation of some hacks on X to account for the drastic imbalance in word frequency (see below)




**Notes**

All of numpy, scipy, and scikit-learn (sklearn) provide some implementation of SVD, but only scipy and sklearn provide an implementation of Truncated SVD, and only sklearn provides an efficient randomized algorithm for calculating large-scale Truncated SVD

- [numpy.linalg.svd](https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html)
- [scipy.linalg.svd](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.svd.html)
- [scipy.linalg.svdvals](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.svdvals.html#scipy.linalg.svdvals)
- [sklearn.decomposition.TruncatedSVD](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html)



<a name='3-2-2'></a>
### 3.2.2 Other tricks: scaling, pearson correlations etc.

several used in [Rohde et al. 2005. An Improved Model of Semantic Similarity Based on Lexical Co-Occurrence.](https://www.cnbc.cmu.edu/~plaut/papers/pdf/RohdeGonnermanPlautSUB-CogSci.COALS.pdf)

- Scaling the counts in the cells can help a lot
    - Problem: function words (the, he, has) are too frequentàsyntax has too much impact. Some fixes:
         - log the frequencies
         - min(X,t), with t ≈ 100
         - Ignore the function words
- Ramped windows that count closer words more than further away words 
- Use Pearson correlations instead of counts, then set negative values to 0


<img src='../images/2-counts-scaling-tricks-effcts.png' width='600' height='300'>


<a name='3-3'></a>
## 3.3 Count based method vs. direct prediction (neural network) method

<img src='../images/2-counts-based-versus-direct-prediction.png' width='600' height='300'>




<a name='4'></a>
# 4. GloVe: Global Vectors for Word Representation

All in [Pennington et. al. 2014. GloVe: Global Vectors for Word Representation](https://nlp.stanford.edu/pubs/glove.pdf)

<a name='4-1'></a>
## 4.1 Basic idea
- Ratios of co-occurrence probabilities can encode meaning components
- This reminds me of [mutual information](https://en.wikipedia.org/wiki/Mutual_information) in information retrieval theory, which takes the logged ratio of P(X, Y) to the product of P(X) and P(Y). Mutual information used to be used as an unsupervised way to segment words for Chinese and find collocations for English (also can see [Manning & Schütze. 1999. Foundations of Statistical Natural Language Processing](https://nlp.stanford.edu/fsnlp/)). 
- Example:


<img src='../images/2-glove-basic-ideas.png' width='600' height='300'>


<a name='4-2'></a>
## 4.2 Mathematical realization 

- Log-bilinear model:

$$w_{i} \cdot w_{j} = \log P(i|j) \tag{8}$$

- With vector differences (expressed in ratio term):


$$w_{x} \cdot (w_{a} - w_{b}) = w_{x} \cdot w_{a}  - w_{x} \cdot w_{b} = \log P(x|a) - \log P(x|b) = \log \frac{P(x|a)}{P(x|b)}$$

In short:


$$w_{x} \cdot (w_{a} - w_{b}) = \log \frac{P(x|a)}{P(x|b)} \tag{9}$$


**objective/loss function**

$$J = \sum_{i,j=1}^{V} f(X_{ij})(w_{i}^{T} \tilde{w_{j}} + b_{i} + \tilde{b_{j}} - \log(X_{ij})))^{2} \tag{10}$$

- $J$ is a squared loss function that aims to minimize the differences between the dot product of $w_{i}$ and $\tilde{w_{j}}$ and their occurrence probability. 
- $b_{i}$ and $\tilde{b_{j}}$ are bias terms for $w_{i}$ and $\tilde{w_{j}}$ respectively. 
- $f(X_{ij})$ is a sumsampling function that looks like the following: 

<img src='../images/2-glove-subsampling-f.png' width='400' height='200'>


- Advantages:
    - Fast training
    - Scalable to huge corpora
    - Good performance even with small corpus and small vectors


<a name='5'></a>
# 5. Evaluation of word vectors

<a name='5-1'></a>
## 5.1 Overview 

- Related to general evaluation in NLP: Intrinsic vs. extrinsic
- Intrinsic:
    - Evaluation on a specific/intermediate subtask
    - Fast to compute
    - Helps to understand that system
    - **Not clear if really helpful unless correlation to real task is established**
- Extrinsic:
    - Evaluationonarealtask
    - Can take a long time to compute accuracy
    - Unclear if the subsystem is the problem or its interaction or other subsystems
    - Unstable metrics: if replacing exactly one subsystem with another, accuracy may just be improved
    

<a name='5-2'></a>
## 5.2 Intrinsic word vector evaluation

<a name='5-2-1'></a>
### 5.2.1 Models


**1-Word analogy test**
- Related stuff: [Mikolov Word2vec word analogy corpus](https://github.com/tmikolov/word2vec/blob/master/questions-words.txt)

<img src='../images/2-intrinsic-eval.png' width='600' height='300'>


**2-Human judgments correlation test**

- Related stuff: http://alfonseca.org/eng/research/wordsim353.html (The link posted on the slide is not accessible.)

<img src='../images/2-intrinsic-eval-2.png' width='600' height='300'>


<a name='5-2-2'></a>
### 5.2.2 Performances of different models

GloVe model seems to perform well on both tests. 

**1-Word analogy test**

<img src='../images/2-word-vectors-models-comp.png' width='600' height='300'>


**2-Human judgments correlation test**

<img src='../images/2-word-vectors-models-comp-2.png' width='600' height='300'>



<a name='5-3'></a>
## 5.3 Extrinsic word vector evaluation

Extrinsic evaluation of word vectors: All subsequent NLP tasks in this class, such as Named Identify Recognition.

<img src='../images/2-extrinsic-eval.png' width='600' height='300'>




<a name='5-4'></a>
## 5.4 Factors that impact the performances  



- Related paper: [Yin & Shen. 2018. On the Dimensionality of Word Embedding.](https://arxiv.org/pdf/1812.04224.pdf)
- The following slides are from the lecture video. The lecture slide is from 2021, so something has been revised compared to that used in the video. 

<img src='../images/2-hyperparam-effects-on-evaluations1.png' width='600' height='300'>

<img src='../images/2-hyperparam-effects-on-evaluations2.png' width='600' height='300'>

<img src='../images/2-hyperparam-effects-on-evaluations3.png' width='600' height='300'>
 

<a name='6'></a>
# 6. Challenges: word sense ambiguity 

<a name='6-1'></a>
## 6.1 Challenges

- Most words have lots of meanings:
    - Especially common words
    - Especially words that have existed for a long time
    - Example:

<img src='../images/2-word-sense-ambiguity-pike.png' width='600' height='300'>


<a name='6-2'></a>
## 6.2 Tentative solutions


- Related paper: [Huang et. al. 2012. Improving Word Representations via Global Context and Multiple Word Prototypes](https://aclanthology.org/P12-1092.pdf)

<img src='../images/2-tentative-solution-to-word-senses.png' width='600' height='300'>



- More deep learning: 

Related paper: [Arora et. al. 2018. Linear Algebraic Structure of Word Senses, with Applications to Polysemy](https://transacl.org/ojs/index.php/tacl/article/viewFile/1346/320)

<img src='../images/2-tentative-solution-to-word-senses2.png' width='600' height='300'>



<a name='7'></a>
# 7. References


- [Course website](http://web.stanford.edu/class/cs224n/index.html)

- [Lecture video](https://www.youtube.com/watch?v=kEMJRjEdNzM) 

- [Lecture slide](http://web.stanford.edu/class/cs224n/slides/cs224n-2021-lecture02-wordvecs2.pdf)

- [Assignment 1](http://web.stanford.edu/class/cs224n/index.html#schedule)

- [Mikolov et. al. (2013a). Efficient estimation of word representations in vector space](https://arxiv.org/pdf/1301.3781.pdf)  
- [Mikolov et. al. (2013b). Distributed representation of words and phrases and their compositionality](https://papers.nips.cc/paper/2013/file/9aa42b31882ec039965f3c4923ce901b-Paper.pdf)
- [Pennington et. al. 2014. GloVe: Global Vectors for Word Representation](https://nlp.stanford.edu/pubs/glove.pdf)
